In [1]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import pandas as pd
import pyasn

%matplotlib inline

In [32]:
asndb = pyasn.pyasn('ipasn_20211008.dat', as_names_file='asnames.json')

In [2]:
databases = ['ec2-18-197-150-189.eu-central-1.compute.db',
'ec2-3-91-76-120.compute-1.db',
'ec2-54-153-52-209.us-west-1.compute.db',
'ec2-13-244-77-73.af-south-1.compute.db',
'ec2-13-208-166-38.ap-northeast-3.compute.db',
'ec2-54-79-89-121.ap-southeast-2.compute.db',
'ec2-54-232-145-115.sa-east-1.compute.db']

connections = []
cursors = []
db_pre_string = './misc-measurements/'

for database in databases:
    conn = sq.connect(f"{db_pre_string}{database}")
    conn.row_factory = sq.Row
    connections.append(conn)
    cursors.append(conn.cursor())

In [3]:
def check_commonalities(index_a, db_a, db_b, table, cols):
    attach_sql = f"""
    ATTACH DATABASE '{db_pre_string}{db_b}' AS db_b;
    """
    cursors[index_a].execute(attach_sql)
    connections[index_a].commit()
    
    detach_sql = """
    DETACH DATABASE db_b;
    """
    
    try:
        cols_string = ', '.join(cols)
        main_sql = f"""
        SELECT {cols_string} FROM {table}
        EXCEPT
        SELECT {cols_string} FROM db_b.{table};
        """
        cursors[index_a].execute(main_sql)
        items = list(cursors[index_a].fetchall())
        
    except:
        cursors[index_a].execute(detach_sql)
        connections[index_a].commit()
    cursors[index_a].execute(detach_sql)
    connections[index_a].commit()
    return items

In [33]:
def create_commonality_df(table, cols):
    indices = []
    data = dict((col, []) for col in cols)
    if 'ip' in cols:
        data['as'] = []
    for index_a, db_a in enumerate(databases):
        for db_b in databases:
            if db_a != db_b:
                items = check_commonalities(index_a, db_a, db_b, table, cols)

                if items:
                    for item in items:
                        indices.append(f"{db_a.split('.')[1].rsplit('-', 1)[0]} - {db_b.split('.')[1].rsplit('-', 1)[0]}")
                        for col in cols:
                            data[col].append(item[col])
                            
                        if 'ip' in cols:
                            asn, prefix = asndb.lookup(item['ip'])
                            data['as'].append(asndb.get_as_name(asn))
                        
    indices = [i.replace('compute', 'us-east') for i in indices]
    return pd.DataFrame(data, index=indices)

In [39]:
# Cross checking 
cols = ['ip', 'draft_version']
table = 'q_versions'
df0 = create_commonality_df(table, cols)
df0

,ip,draft_version,as
eu-central - af-south,41.223.53.120,doq,"CITYNET, EG"
eu-central - af-south,73.79.210.148,doq-i02,"COMCAST-7922, US"
eu-central - af-south,82.65.248.221,doq-i02,"PROXAD, FR"
eu-central - ap-northeast,114.204.32.239,doq-i02,"SKB-AS SK Broadband Co Ltd, KR"
eu-central - ap-northeast,82.65.248.221,doq-i02,"PROXAD, FR"
eu-central - ap-southeast,82.65.248.221,doq-i02,"PROXAD, FR"
eu-central - sa-east,114.204.32.239,doq-i02,"SKB-AS SK Broadband Co Ltd, KR"
eu-central - sa-east,82.65.248.221,doq-i02,"PROXAD, FR"
us-east - eu-central,103.254.108.74,doq-i02,SUNHK-DATA-AS-AP Sun Network Hong Kong Limited...
us-east - eu-central,35.198.139.194,doq-i02,"GOOGLE, US"


In [ ]:
cols = ['ip', 'port', 'support']
table = 'q0_rtt_supports'
df1 = create_commonality_df(table, cols)
df1

In [36]:
cols = ['ip', 'port', 'support']
table = 'fast_open_supports'
df2 = create_commonality_df(table, cols)
df2

,ip,as
eu-central - us-east,181.214.88.17,"ZHUSUP-AS, KG"
eu-central - us-west,181.214.88.17,"ZHUSUP-AS, KG"
eu-central - af-south,73.79.210.148,"COMCAST-7922, US"
eu-central - af-south,82.65.248.221,"PROXAD, FR"
eu-central - ap-northeast,114.204.32.239,"SKB-AS SK Broadband Co Ltd, KR"
eu-central - ap-northeast,82.65.248.221,"PROXAD, FR"
eu-central - ap-southeast,82.65.248.221,"PROXAD, FR"
eu-central - sa-east,114.204.32.239,"SKB-AS SK Broadband Co Ltd, KR"
eu-central - sa-east,82.65.248.221,"PROXAD, FR"
us-east - eu-central,92.255.176.201,"ERTH-OMSK-AS, RU"


In [37]:
cols = ['ip', 'port', 'support']
table = 'e_dns0'
df3 = create_commonality_df(table, cols)
df3

,ip,as
eu-central - us-east,185.229.251.250,"ON-LINE-DATA Server location - Netherlands, Dr..."
eu-central - us-east,210.16.120.48,"HOSTUS-GLOBAL-AS HostUS, HK"
eu-central - us-east,47.107.67.12,CNNIC-ALIBABA-CN-NET-AP Hangzhou Alibaba Adver...
eu-central - us-east,5.253.114.91,"CLOUVIDER Clouvider - Global ASN, GB"
eu-central - us-east,82.156.3.118,CNNIC-TENCENT-NET-AP Shenzhen Tencent Computer...
...,...,...
sa-east - us-east,130.193.38.204,"YANDEXCLOUD, RU"
sa-east - us-west,130.193.38.204,"YANDEXCLOUD, RU"
sa-east - af-south,121.196.146.239,CNNIC-ALIBABA-CN-NET-AP Hangzhou Alibaba Adver...
sa-east - af-south,43.128.23.18,"TENCENT-NET-AP-CN Tencent Building, Kejizhongy..."


In [38]:
cols = ['ip', 'protocol', 'port']
table = 'certificates'
df4 = create_commonality_df(table, cols)
df4

,ip,as
eu-central - us-east,139.162.7.92,"LINODE-AP Linode, LLC, US"
eu-central - us-west,139.162.7.92,"LINODE-AP Linode, LLC, US"
eu-central - us-west,140.238.41.43,"ORACLE-BMC-31898, US"
eu-central - af-south,139.162.7.92,"LINODE-AP Linode, LLC, US"
eu-central - af-south,73.79.210.148,"COMCAST-7922, US"
...,...,...
sa-east - us-east,172.104.94.52,"LINODE-AP Linode, LLC, US"
sa-east - us-west,140.238.41.43,"ORACLE-BMC-31898, US"
sa-east - us-west,172.104.94.52,"LINODE-AP Linode, LLC, US"
sa-east - af-south,73.79.210.148,"COMCAST-7922, US"


In [13]:
for cur in cursors:
    cur.close()
for conn in connections:
    conn.close()